# 今天我們要來講解下方這個爬取月報的 Function 囉！

## 雖然它長得有點可怕，不過大家不用緊張，我們會在下方一一為各位拆解說明！

## (大家可以先跳過第一個 cell 喔～～)

In [24]:
### import requests
import pandas as pd
from io import StringIO

#定義月報爬蟲的function
def crawl_monthly_report(date):
    
    #指定爬取月報的網址（指定特定年份和月份）
    url = 'http://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(date.year - 1911)+'_'+str(date.month)+'_0.html'
    if date.year - 1991 <= 98: #民國98年以前的月報爬取網址不一樣
        url = 'http://mops.twse.com.tw/nas/t21/sii/t21sc03_'+str(date.year - 1911)+'_'+str(date.month)+'.html'
    
    # 偽瀏覽器（讓request能夠偽裝成瀏覽器）
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    
    # 下載該年月的網站，並用pandas轉換成 dataframe
    r = requests.get(url, headers)
    r.encoding = 'big5'# 讓pandas可以讀取中文（測試看看，假如不行讀取中文，就改成 'utf-8'）
    
    # 這裡使用 try 和 except 是因為 dataframe 有可能無法讀取 html 檔，在這樣的情況下，程式就會跑出 None，而不會因為讀取不了而出現 error
    try:
        html_df = pd.read_html(StringIO(r.text))
    except:
        return None

    # 現在開始整理一下資料
    # 我們要先取出從網頁下載下來的 table，但是因為月報網頁的格式會因為日期而有所不同
    # (有些是一個大的總表加上一些重複的小圖表，有些沒有大圖表，只有分散的小圖表)
    # 所以下載下來的圖表需要先經過一些判斷，來決定我們需要取圖表的是哪個！
    
    # 首先，我們先看看下載下來的第一個圖表的 row 是否大於 500 行，來判斷第一個圖表是不是大總表
    if len(html_df[0]) > 500:
        
        # 如果是大圖表的話，它就是我們所需要的月報
        df = html_df[0].copy()
        
    # 如果不是的話，我們就需要把所有的小圖表拼湊成我們需要的月報
    else:
        
        # 我們先判斷這些小圖表的 column 是否小於等於 11，然後把這些 column 小於 11 的小圖表全部加在一起(.concat) 就是大總表了！
        df = pd.concat([df for df in html_df if df.shape[1] <= 11])

    # 用 list(range(10)) 取 [0,1,2,...,9]，用來選取第0到9個 column
    df = df[list(range(0,10))]
    
    # 首先我們可以先取出第0欄為「公司代號」的 rows (df[0] == '公司代號')
    column_index = df.index[(df[0] == '公司代號')][0]
    
    # 選取 column_index 裡面任意一條 row 當作 column 的名稱 （因為這裡所有的 row 都長的一樣）
    df.columns = df.iloc[column_index]
    
    # 將 df 中的當月營收用 .to_numeric 變成數字，再把其中不能變成數字的部分以 NaN 取代（errors='coerce'）
    df['當月營收'] = pd.to_numeric(df['當月營收'], errors='coerce')
    
    # 以 .isnull() 檢查是否為 NaN，再取其否定「～」的行數作為新的 df
    df = df[~df['當月營收'].isnull()]
    
    # 取「公司代號」中，不是「合計」的行數
    df = df[df['公司代號'] != '合計']
    
    # 找出下個月的月報出爐日（每個月的10號）
    next_month = datetime.date(date.year + int(date.month / 12), ((date.month % 12) + 1), 10)
    df['date'] = pd.to_datetime(next_month)

    # 重新命名「公司代號」為「stock_id」
    df = df.rename(columns={'公司代號':'stock_id'})
    
    # 指定 df 的 indexes
    df = df.set_index(['stock_id', 'date'])
    
    # 將 df 中的所有字串轉成數值，並且把其中沒有 NaN 的行數取出
    df = df.apply(lambda s:pd.to_numeric(s, errors='coerce'))
    df = df[df.columns[df.isnull().all() == False]]
    
    return df

# 實際使用這個 function 試試看！
import datetime
df = crawl_monthly_report(datetime.date(2018,2,1))
df.head()

,4,當月營收,上月營收,去年當月營收,上月比較增減(%),去年同月增減(%),當月累計營收,去年累計營收,前期比較增減(%)
stock_id,date,,,,,,,,
1101,2018-03-10,4811678.0,9617393,5964991,-49.96,-19.33,14429071,12096661,19.28
1102,2018-03-10,3408539.0,6769986,3049461,-49.65,11.77,10178525,7211591,41.14
1103,2018-03-10,125590.0,172094,171513,-27.02,-26.77,297684,404170,-26.34
1104,2018-03-10,268120.0,460482,308357,-41.77,-13.04,728601,672222,8.38
1108,2018-03-10,197481.0,320340,232100,-38.35,-14.91,517821,548873,-5.65


# 先來讀資料！

In [25]:
import requests

# 指定爬取月報的網址
url = 'http://mops.twse.com.tw/nas/t21/sii/t21sc03_106_1_0.html'
# 抓取網頁
r = requests.get(url)

In [26]:
from io import StringIO
import pandas as pd

# 讓pandas可以讀取中文（測試看看，假如不行讀取中文，就改成 'utf-8'）
r.encoding = 'big5'
# 把剛剛下載下來的網頁的 html 文字檔，利用 StringIO() 包裝成一個檔案給 pandas 讀取
dfs = pd.read_html(StringIO(r.text))

# 來處理一下資料

In [29]:
# 取出剛剛下載下來的 html 檔案裡面的第一個圖表，通常我們下載下來的第一個圖表 (dfs[0]) 就是月報的總表
df = dfs[0]

# --------------------------------------------- #
# 我們用 iloc 來取出所有的 rows  和 前十個 columns  #
# --------------------------------------------- #

# 在 [:,:10] 中，逗點前面指定 row 的 id，逗點後面用來指定 columns 的 id
# 用 「:」 代表，這裡的冒號前後都沒有放數字就代表了我們取頭到尾，「:10」，代表我們從第0個開始取到第9個
df = df.iloc[:,:10] 

# df = df[list(range(10))] <----影片中的寫法，可以取代上面那行，其中 list(range(10)) 是 [0,1,2,...,9]，用來選取第0到9個 column

# --------------------- #
# 設定正確的 columns 名稱 #
# --------------------- # 

# 首先我們可以先取出第0欄為「公司代號」的 rows (df[0] == '公司代號') 
column_name = df[df[0] == '公司代號']

# 選取 column_name 裡面任意一條 row 當作 column 的名稱 （因為這裡所有的 row 都長的一樣）
df.columns = column_name.iloc[0]

# 將 df 中的當月營收用 .to_numeric 變成數字，再把其中不能變成數字的部分以 NaN 取代（errors='coerce'）
df['當月營收'] = pd.to_numeric(df['當月營收'], errors='coerce')
# 再把當月營收中，出現 NaN 的 row 用 .dropna 整行刪除
df = df.dropna(subset=['當月營收'])

#df = df.loc[~pd.to_numeric(df['當月營收'], errors='coerce').isnull()] ---->影片中的寫法，可以取代上面兩行（以 .isnull() 檢查是否為 NaN，再取其否定「～」的行數作為新的 df）

# 刪除「公司代號」中出現「合計」的行數，其中「～」是否定的意思
df = df.loc[~(df['公司代號'] == '合計')]

# 將「公司代號」與「公司名稱」共同列為 df 的 indexes
df = df.set_index(['公司代號', '公司名稱'])

# 最後，將 df 中的所有字串轉成數值
df = df.apply(pd.to_numeric)
df.head()

,4,當月營收,上月營收,去年當月營收,上月比較增減(%),去年同月增減(%),當月累計營收,去年累計營收,前期比較增減(%)
公司代號,公司名稱,,,,,,,,
1101,台泥,6131670.0,9418581,6575590,-34.89,-6.75,6131670,6575590,-6.75
1102,亞泥,4162130.0,6023954,5066062,-30.90,-17.84,4162130,5066062,-17.84
1103,嘉泥,232657.0,382981,197328,-39.25,17.90,232657,197328,17.90
1104,環球水泥,363865.0,463842,373311,-21.55,-2.53,363865,373311,-2.53
1108,幸福水泥,316773.0,345573,348266,-8.33,-9.04,316773,348266,-9.04


In [31]:
# ----------- #
# 存取 csv 檔  #
# ----------- #

# 把 df 存成 csv 檔，並且命名為「test.csv」，指定用「utf_8_sig」編碼
df.to_csv('test.csv', encoding='utf_8_sig')

# 讀取名為「test.csv」的 csv 檔，並且指定其中欄位名稱為「公司代號」與「公司名稱」作為 df 的 indexes
df = pd.read_csv('test.csv', index_col=['公司代號','公司名稱'])
df.head()

,,當月營收,上月營收,去年當月營收,上月比較增減(%),去年同月增減(%),當月累計營收,去年累計營收,前期比較增減(%)
公司代號,公司名稱,,,,,,,,
1101,台泥,6131670.0,9418581,6575590,-34.89,-6.75,6131670,6575590,-6.75
1102,亞泥,4162130.0,6023954,5066062,-30.90,-17.84,4162130,5066062,-17.84
1103,嘉泥,232657.0,382981,197328,-39.25,17.90,232657,197328,17.90
1104,環球水泥,363865.0,463842,373311,-21.55,-2.53,363865,373311,-2.53
1108,幸福水泥,316773.0,345573,348266,-8.33,-9.04,316773,348266,-9.04


In [42]:
# --------------- #
# 存取 sqlite3 檔  #
# --------------- #

import sqlite3

# 把 df 存成名為「monthly_report」的 sqlite3 檔，其中 conn 是與 database 的連結
conn = sqlite3.connect('test.sqlite3')
df.to_sql('monthly_report', conn, if_exists='replace')

# 讀取 sqlite3 中名為「monthly_report」的 table，並且指定其中欄位名稱為「公司代號」與「公司代號」作為 df 的 indexes
df = pd.read_sql('select * from monthly_report', conn, index_col=['公司代號','公司名稱'])
df.head()




,,當月營收,上月營收,去年當月營收,上月比較增減(%),去年同月增減(%),當月累計營收,去年累計營收,前期比較增減(%)
公司代號,公司名稱,,,,,,,,
1101,台泥,6131670,9418581,6575590,-34.89,-6.75,6131670,6575590,-6.75
1102,亞泥,4162130,6023954,5066062,-30.90,-17.84,4162130,5066062,-17.84
1103,嘉泥,232657,382981,197328,-39.25,17.90,232657,197328,17.90
1104,環球水泥,363865,463842,373311,-21.55,-2.53,363865,373311,-2.53
1108,幸福水泥,316773,345573,348266,-8.33,-9.04,316773,348266,-9.04
